In [1]:
from bs4 import BeautifulSoup as bs
import requests
url = 'http://sofifa.com/players?offset=80'

In [2]:
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup

In [3]:
def find_top_players(soup):
    table = soup.find('table', {'class': 'table table-hover persist-area'})
    tbody = table.find('tbody')
    all_a = tbody.find_all('a', {'class': ''})
    return(['http://sofifa.com' + player['href'] for player in all_a if player['href'][0:8] == '/player/'])

In [8]:
from bs4 import BeautifulSoup as bs
import requests
import re



fifa_stats = ['Crossing', 'Finishing', 'Heading Accuracy',
              'Short Passing', 'Volleys', 'Dribbling', 'Curve',
              'Free Kick Accuracy', 'Long Passing', 'Ball Control',
              'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
              'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
              'Long Shots', 'Aggression', 'Interceptions', 'Positioning',
              'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle',
              'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking',
              'GK Positioning', 'GK Reflexes']


def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup


def find_top_players(soup):
    final_details = {}
    table = soup.find('table', {'class': 'table table-hover persist-area'})
    tbody = table.find('tbody')
    all_a = tbody.find_all('a')
    #print(all_a)
    result=[]
    for player in all_a:
        #print(player['href'])
        final_details['short_name'] = player.text
        #print(final_details)
        if player['href'][0:8] == '/player/':
            final_details.update(player_all_details('http://sofifa.com' + player['href']))
        dict2 = final_details.copy()
        result.append(dict2)
    return result


def find_player_info(soup):
    player_data = {}
    player_data['image'] = soup.find('img')['data-src']
    player_data['full_name'] = soup.find('h1').text.split(' (')[0]
    span = soup.find('span', attrs={'class': None}).text.strip()
    #span = soup.find('span').text.strip()
    #print(soup)
    #print(span)
    dob = re.search('(\(.*)\)', span).group(0)
    player_data['dob'] = dob.replace('(', '').replace(')', '')
    #print(span)
    infos = span.replace(dob + ' ', '').split(' ')
    #infos = span.split('Age')
    infos.append(infos[-1])
    infos[-2] = infos[-3]
    infos[-3] = infos[-4]
    infos[-4] = infos[-5][-3:-1]
    infos[-4] = infos[-4]+infos[-5][-1]
    infos[-5] = infos[-5][:-3]
    #print(infos)
    player_data['pref_pos'] = infos[:infos.index('Age')]
    #print(player_data['pref_pos'])
    player_data['age'] = int(infos[infos.index('Age') + 1: -2][0])
    player_data['height'] = int((infos[infos.index('Age') + 2: -1][0]).replace('cm', ''))
    player_data['weight'] = int((infos[infos.index('Age') + 3:][0]).replace('kg', ''))
    return(player_data)


def find_player_stats(soup):
    player_data = {}
    info = re.findall('\d+', soup.text)
    player_data['rating'] = int(info[0])
    player_data['potential'] = int(info[1])
    player_data['value'] = int(info[2])
    player_data['wage'] = int(info[3])
    return(player_data)

def find_player_stats(soup):
    player_data = {}
    info = re.findall('\d+', soup.text)
    #print(info[4])
    player_data['rating'] = int(info[0])
    player_data['potential'] = int(info[1])
    player_data['value'] = int(info[2])
    if len(info)== 5:
        player_data['wage'] = int(info[4])*1000
    else:
        player_data['wage'] = int(info[3])*1000
    return(player_data)

def find_player_secondary_info(soup):
    player_data = {}
    #print(soup)
    player_data['preff_foot'] = soup.find('label', text='Preferred Foot')\
        .parent.contents[2].strip('\n ')
    player_data['club'] = soup.find_all('ul')[1].find('a').text
    player_data['club_pos'] = soup.find('label', text='Position')\
        .parent.find('span').text
    player_data['club_jersey'] = soup.find('label', text='Jersey Number')\
        .parent.contents[2].strip('\n ')
    if soup.find('label', text='Joined'):
        player_data['club_joined'] = soup.find('label', text='Joined')\
            .parent.contents[2].strip('\n ')
    if soup.find('label', text='Contract Valid Until'):
        player_data['contract_valid'] = soup.find('label', text='Contract Valid Until').parent.contents[2].strip('\n ')
    if len(soup.find_all('ul')) > 2:
        player_data['country'] = soup.find_all('ul')[2].find('a').text
    return(player_data)


def find_fifa_info(soup):
    player_data = {}
    #print(soup[4])
    #divs_without_skill = soup[1].find_all('div', {'class': 'column col-3 mb-20'})[:3]
    divs_without_skill = soup[4].find_all('div', {'class': 'column col-3 mb-20'})[:3]
    #print(divs_without_skill)
    more_lis = [div.find_all('li') for div in divs_without_skill]
    lis = soup[0].find_all('li') + more_lis[0]
    #print(lis)
    for li in lis:
        for stats in fifa_stats:
            if stats in li.text:
                #print(stats.replace(' ', '_').lower())
                #print(li.text.split(' ')[0]).replace('\n', '')
                player_data[stats.replace(' ', '_').lower()] = int((li.text.split(' ')[0]).replace('\n', ''))
    #traits = soup[1].find('h4', text='Traits')
    traits = soup[4].find('h5', text='Traits')
    #print(traits)
    if traits:
        #print(traits.parent.find_all('li'))
        #player_data['traits'] = [li.text.replace('\xa0', '') for li in traits.parent.next_sibling.next_sibling.find_all('li')]
        player_data['traits'] = [li.text.replace('\xa0', '') for li in traits.parent.find_all('li')]
    specialities = soup[1].find('h4', text='Specialities')
    if specialities:
        #print('hello')
        player_data['specialities'] = [li.text.replace('\xa0', '') for li in
            specialities.parent.next_sibling.next_sibling.find_all('li')]
    return(player_data)

def playerskills(url):
    all_details = {}
    soup = soup_maker(url)
    a = soup.find_all('div', {'class': 'mb-20'})
    li = []
    k=0
    for i in a:
        b = i.find_all('li')
        for j in b:
            li.append(j.text)
    dict1={}
    j=0
    for i in li:
        if j>=34:
            break
        a=i.split()
        if len(a)==2:
            dict1.update({str(a[1]):int(a[0])})
        else:
            key=str(a[-2])+str(a[-1])
            dict1.update({key:int(a[0])})
        j+=1
        
    return dict1


def player_all_details(url):
    all_details = {}
    soup = soup_maker(url)
    player_info = soup.find('div', {'class': 'player'})
    all_details.update(find_player_info(player_info))   
    player_stats = soup.find('div', {'class': 'stats'})
    all_details.update(find_player_stats(player_stats))
    secondary_info = soup.find('div', {'class': 'teams'})
    all_details.update(find_player_secondary_info(secondary_info))
    fifa_info = soup.find_all('div', {'class': 'columns'})
    all_details.update(find_fifa_info(fifa_info))
    dict1=playerskills(url)
    print(dict1)
    all_details.update(dict1)
    return(all_details)

In [5]:
soup = soup_maker(url)
result=find_top_players(soup)

{'Crossing': 77, 'Finishing': 95, 'HeadingAccuracy': 71, 'ShortPassing': 88, 'Volleys': 86, 'Dribbling': 97, 'Curve': 90, 'FKAccuracy': 92, 'LongPassing': 87, 'BallControl': 96, 'Acceleration': 92, 'SprintSpeed': 87, 'Agility': 90, 'Reactions': 95, 'Balance': 95, 'ShotPower': 85, 'Jumping': 67, 'Stamina': 73, 'Strength': 59, 'LongShots': 88, 'Aggression': 48, 'Interceptions': 22, 'Positioning': 93, 'Vision': 92, 'Penalties': 75, 'Composure': 97, 'Marking': 13, 'StandingTackle': 28, 'SlidingTackle': 26, 'GKDiving': 6, 'GKHandling': 11, 'GKKicking': 15, 'GKPositioning': 14, 'GKReflexes': 8}
{'Crossing': 85, 'Finishing': 94, 'HeadingAccuracy': 88, 'ShortPassing': 83, 'Volleys': 88, 'Dribbling': 91, 'Curve': 81, 'FKAccuracy': 76, 'LongPassing': 77, 'BallControl': 93, 'Acceleration': 89, 'SprintSpeed': 91, 'Agility': 89, 'Reactions': 96, 'Balance': 63, 'ShotPower': 94, 'Jumping': 95, 'Stamina': 92, 'Strength': 80, 'LongShots': 92, 'Aggression': 63, 'Interceptions': 29, 'Positioning': 95, 'V

{'Crossing': 13, 'Finishing': 11, 'HeadingAccuracy': 15, 'ShortPassing': 29, 'Volleys': 13, 'Dribbling': 12, 'Curve': 13, 'FKAccuracy': 14, 'LongPassing': 26, 'BallControl': 16, 'Acceleration': 43, 'SprintSpeed': 60, 'Agility': 67, 'Reactions': 85, 'Balance': 49, 'ShotPower': 22, 'Jumping': 76, 'Stamina': 41, 'Strength': 78, 'LongShots': 12, 'Aggression': 34, 'Interceptions': 19, 'Positioning': 11, 'Vision': 55, 'Penalties': 11, 'Composure': 55, 'Marking': 14, 'StandingTackle': 12, 'SlidingTackle': 18, 'GKDiving': 85, 'GKHandling': 91, 'GKKicking': 77, 'GKPositioning': 87, 'GKReflexes': 88}
{'Crossing': 14, 'Finishing': 14, 'HeadingAccuracy': 13, 'ShortPassing': 33, 'Volleys': 12, 'Dribbling': 13, 'Curve': 19, 'FKAccuracy': 20, 'LongPassing': 35, 'BallControl': 23, 'Acceleration': 46, 'SprintSpeed': 52, 'Agility': 61, 'Reactions': 80, 'Balance': 45, 'ShotPower': 36, 'Jumping': 68, 'Stamina': 38, 'Strength': 70, 'LongShots': 17, 'Aggression': 23, 'Interceptions': 15, 'Positioning': 13, 

{'Crossing': 55, 'Finishing': 42, 'HeadingAccuracy': 92, 'ShortPassing': 79, 'Volleys': 47, 'Dribbling': 53, 'Curve': 49, 'FKAccuracy': 51, 'LongPassing': 70, 'BallControl': 76, 'Acceleration': 64, 'SprintSpeed': 70, 'Agility': 63, 'Reactions': 85, 'Balance': 58, 'ShotPower': 67, 'Jumping': 89, 'Stamina': 67, 'Strength': 80, 'LongShots': 43, 'Aggression': 88, 'Interceptions': 88, 'Positioning': 48, 'Vision': 52, 'Penalties': 50, 'Composure': 82, 'Marking': 87, 'StandingTackle': 86, 'SlidingTackle': 89, 'GKDiving': 6, 'GKHandling': 8, 'GKKicking': 15, 'GKPositioning': 5, 'GKReflexes': 15}
{'Crossing': 70, 'Finishing': 85, 'HeadingAccuracy': 89, 'ShortPassing': 78, 'Volleys': 85, 'Dribbling': 80, 'Curve': 77, 'FKAccuracy': 76, 'LongPassing': 50, 'BallControl': 82, 'Acceleration': 75, 'SprintSpeed': 78, 'Agility': 81, 'Reactions': 91, 'Balance': 59, 'ShotPower': 86, 'Jumping': 88, 'Stamina': 92, 'Strength': 75, 'LongShots': 79, 'Aggression': 84, 'Interceptions': 48, 'Positioning': 93, 'Vi

{'Crossing': 77, 'Finishing': 87, 'HeadingAccuracy': 79, 'ShortPassing': 77, 'Volleys': 86, 'Dribbling': 79, 'Curve': 78, 'FKAccuracy': 74, 'LongPassing': 64, 'BallControl': 82, 'Acceleration': 94, 'SprintSpeed': 95, 'Agility': 77, 'Reactions': 86, 'Balance': 70, 'ShotPower': 81, 'Jumping': 77, 'Stamina': 78, 'Strength': 76, 'LongShots': 78, 'Aggression': 43, 'Interceptions': 48, 'Positioning': 89, 'Vision': 77, 'Penalties': 76, 'Composure': 84, 'Marking': 28, 'StandingTackle': 25, 'SlidingTackle': 36, 'GKDiving': 6, 'GKHandling': 9, 'GKKicking': 15, 'GKPositioning': 9, 'GKReflexes': 9}
{'Crossing': 69, 'Finishing': 34, 'HeadingAccuracy': 85, 'ShortPassing': 75, 'Volleys': 53, 'Dribbling': 67, 'Curve': 56, 'FKAccuracy': 31, 'LongPassing': 80, 'BallControl': 70, 'Acceleration': 71, 'SprintSpeed': 79, 'Agility': 58, 'Reactions': 82, 'Balance': 53, 'ShotPower': 79, 'Jumping': 75, 'Stamina': 74, 'Strength': 91, 'LongShots': 58, 'Aggression': 82, 'Interceptions': 81, 'Positioning': 47, 'Vis

{'Crossing': 86, 'Finishing': 55, 'HeadingAccuracy': 76, 'ShortPassing': 81, 'Volleys': 68, 'Dribbling': 82, 'Curve': 78, 'FKAccuracy': 59, 'LongPassing': 71, 'BallControl': 81, 'Acceleration': 84, 'SprintSpeed': 88, 'Agility': 79, 'Reactions': 86, 'Balance': 79, 'ShotPower': 80, 'Jumping': 79, 'Stamina': 93, 'Strength': 80, 'LongShots': 72, 'Aggression': 82, 'Interceptions': 82, 'Positioning': 70, 'Vision': 70, 'Penalties': 60, 'Composure': 82, 'Marking': 78, 'StandingTackle': 84, 'SlidingTackle': 84, 'GKDiving': 7, 'GKHandling': 7, 'GKKicking': 9, 'GKPositioning': 12, 'GKReflexes': 5}
{'Crossing': 79, 'Finishing': 87, 'HeadingAccuracy': 82, 'ShortPassing': 81, 'Volleys': 84, 'Dribbling': 75, 'Curve': 81, 'FKAccuracy': 59, 'LongPassing': 68, 'BallControl': 81, 'Acceleration': 76, 'SprintSpeed': 77, 'Agility': 76, 'Reactions': 91, 'Balance': 72, 'ShotPower': 77, 'Jumping': 81, 'Stamina': 90, 'Strength': 67, 'LongShots': 77, 'Aggression': 57, 'Interceptions': 58, 'Positioning': 93, 'Vis

{'Crossing': 76, 'Finishing': 76, 'HeadingAccuracy': 38, 'ShortPassing': 83, 'Volleys': 64, 'Dribbling': 87, 'Curve': 68, 'FKAccuracy': 61, 'LongPassing': 67, 'BallControl': 87, 'Acceleration': 94, 'SprintSpeed': 92, 'Agility': 91, 'Reactions': 82, 'Balance': 92, 'ShotPower': 70, 'Jumping': 60, 'Stamina': 81, 'Strength': 67, 'LongShots': 70, 'Aggression': 34, 'Interceptions': 26, 'Positioning': 86, 'Vision': 76, 'Penalties': 63, 'Composure': 76, 'Marking': 48, 'StandingTackle': 58, 'SlidingTackle': 54, 'GKDiving': 15, 'GKHandling': 12, 'GKKicking': 12, 'GKPositioning': 15, 'GKReflexes': 9}
{'Crossing': 72, 'Finishing': 86, 'HeadingAccuracy': 75, 'ShortPassing': 86, 'Volleys': 81, 'Dribbling': 87, 'Curve': 80, 'FKAccuracy': 67, 'LongPassing': 75, 'BallControl': 87, 'Acceleration': 78, 'SprintSpeed': 79, 'Agility': 82, 'Reactions': 86, 'Balance': 82, 'ShotPower': 81, 'Jumping': 83, 'Stamina': 90, 'Strength': 79, 'LongShots': 76, 'Aggression': 71, 'Interceptions': 58, 'Positioning': 86, '

In [7]:
result[-1]

{'Acceleration': 62,
 'Aggression': 91,
 'Agility': 69,
 'Balance': 62,
 'BallControl': 69,
 'Composure': 84,
 'Crossing': 52,
 'Curve': 50,
 'Dribbling': 49,
 'FKAccuracy': 39,
 'Finishing': 54,
 'GKDiving': 12,
 'GKHandling': 5,
 'GKKicking': 8,
 'GKPositioning': 11,
 'GKReflexes': 12,
 'HeadingAccuracy': 85,
 'Interceptions': 86,
 'Jumping': 87,
 'LongPassing': 72,
 'LongShots': 56,
 'Marking': 83,
 'Penalties': 45,
 'Positioning': 51,
 'Reactions': 78,
 'ShortPassing': 75,
 'ShotPower': 69,
 'SlidingTackle': 85,
 'SprintSpeed': 64,
 'Stamina': 76,
 'StandingTackle': 86,
 'Strength': 83,
 'Vision': 48,
 'Volleys': 57,
 'age': 29,
 'aggression': 91,
 'club': '',
 'club_jersey': '\r\n30',
 'club_joined': '\r\nAug 20, 2015',
 'club_pos': 'LCB',
 'composure': 84,
 'contract_valid': '\r\n2022',
 'country': '',
 'dob': 'Feb 12, 1988',
 'full_name': 'N. Otamendi',
 'height': 183,
 'image': 'https://cdn.sofifa.org/1x/18/players/192366.png',
 'interceptions': 86,
 'penalties': 45,
 'position

In [136]:

def playerskills(url):
    all_details = {}
    soup = soup_maker(url)
    a = soup.find_all('div', {'class': 'mb-20'})
    li = []
    k=0
    for i in a:
        b = i.find_all('li')
        for j in b:
            li.append(j.text)
    dict1={}
    j=0
    for i in li:
        if j>=34:
            break
        a=i.split()
        if len(a)==2:
            dict1.update({str(a[1]):int(a[0])})
        else:
            key=str(a[-2])+str(a[-1])
            dict1.update({key:int(a[0])})
        j+=1
        
    return dict1,li